In [0]:
from pyspark.sql.types import *
customer_schema_change = StructType(
    fields = [
        StructField('customer_id',IntegerType()),
        StructField('customer_name',StringType()),
        StructField('date_of_birth',DateType()),
        StructField('telephone',StringType()),
        StructField('email',StringType()),
        StructField('Member_since',DateType()),
        StructField('Created_timestamp',TimestampType()),

    ]
)

In [0]:

customer_schema_change_df = (spark.readStream
                             .format('cloudFiles')
                             .option("cloudFiles.format", "json")
                             .option("cloudFiles.inferColumnTypes", "true")
                             .option("cloudFiles.mergeSchema", "true")
                             .schema(customer_schema_change)
                            .load("/Volumes/gizmobox/landing/operational_data/customers_stream/")

)

In [0]:
from pyspark.sql.functions import col, current_timestamp, row_number, split
customer_transform_schema_change = (
    customer_schema_change_df
    .withColumn("fname", split(col("customer_name"), " ").getItem(0))
    .withColumn("lname", split(col("customer_name"), " ").getItem(1))
    .withColumn("created_time", current_timestamp())
    .filter(col("customer_id").isNotNull())
)



In [0]:
stream_query = (customer_transform_schema_change.writeStream
                .format("delta")
                .option("checkpointLocation","/Volumes/gizmobox/landing/operational_data/customers_stream/checkpoint_stream")
                .toTable("gizmobox.bronze.customer_autoloader")
                
                
                )